In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install -q transformers datasets sentencepiece

!pip uninstall -y -q transformers accelerate
!pip install -q transformers accelerate

!pip install -q rouge-score

!pip install -q torchmetrics

!pip install -q git+https://github.com/huggingface/accelerate

!git clone -q https://github.com/huggingface/transformers.git
!cd transformers
!pip install  -q -e.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 6.6 MB/s eta 0:00:00


In [ ]:
# model_name_or_path = 'google/mt5-small'
model_name_or_path = '/content/drive/MyDrive/mt5-pquad/checkpoint-11764'
from transformers import AutoTokenizer, MT5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = MT5ForConditionalGeneration.from_pretrained(model_name_or_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pquad

In [ ]:
from datasets import concatenate_datasets, load_dataset

train_data = load_dataset("Gholamreza/pquad", split="train")
valid_data = load_dataset("Gholamreza/pquad", split="validation")
test_data = load_dataset("Gholamreza/pquad", split="test")

concat_dataset = concatenate_datasets([train_data, valid_data, test_data])
dataset=concat_dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/63994 [00:00<?, ? examples/s]

/root/.cache/huggingface/datasets/downloads/e49d5f650d69a5999fe6ceb4438a023cccdcf3e6519abc4dabce736f91595591


Generating validation split:   0%|          | 0/7976 [00:00<?, ? examples/s]

/root/.cache/huggingface/datasets/downloads/ea42ddfa9db6f39bc3249a878c853a6f6b466f6217a360bbb8afbac9410d84cc


Generating test split:   0%|          | 0/8002 [00:00<?, ? examples/s]

/root/.cache/huggingface/datasets/downloads/d6ba3b80ff2a6d0333454fac286694b5e777518ea141e0dcd7c0558b71624882


In [ ]:
dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 79972
})

In [ ]:
# dataset = small_dataset

del concat_dataset, train_data, valid_data, test_data

small_dataset= dataset.filter(lambda example: len(example['answers']['text'])>0)
# small_dataset= small_dataset.filter(lambda example: len(example['context'])<600)
small_dataset= small_dataset.filter(lambda example: 'چرا' not in example['context'])

split_data = small_dataset.train_test_split(test_size=0.2, seed= 42, shuffle=False)

dataset = split_data

del split_data

Filter:   0%|          | 0/79972 [00:00<?, ? examples/s]

Filter:   0%|          | 0/60356 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 47054
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11764
    })
})

In [ ]:
def format_dataset(example):
  return {'input': 'answer: ' + example['answers']['text'][0] + ' context: ' + example['context'], 'target': example['question']}


dataset = dataset.map(format_dataset, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/47054 [00:00<?, ? examples/s]

Map:   0%|          | 0/11764 [00:00<?, ? examples/s]

In [ ]:
test_data = dataset['test']

test_data[0]

{'input': 'answer: باید بخشی را که می\u200cخواهد در آن قرار گیرد انتخاب کند context: برای نامزدی عضویت در آکادمی رقابت بسیار است با این وجود، فرد نامزد باید دعوت شده باشد یا اینکه یکی از اعضا او را معرفی کند. آکادمی نام اعضای خود را به\u200cطور عمومی آشکار نمی\u200cکند، گرچه در مطبوعات گذشته نام مدعوین برای عضویت در آکادمی انتشار یافته بود. در صورتی که کسی که هنوز به عضویت در آکادمی در نیامده در طول یک سال برای عضویت در بیش از یک بخش نامزد شده باشد، در زمانی که عضویت را می\u200cپذیرد باید بخشی را که می\u200cخواهد در آن قرار گیرد انتخاب کند. تعداد اعضای رأی\u200cدهندهٔ آکادمی اسکار بیش از ۶۰۰۰ نفر است.',
 'target': 'زمانی که نامزد عضویت را می\u200cپذیرد چه کاری انجام می\u200cدهد؟'}

In [ ]:
prompt = "answer: اصفهان context: نگار در اصفهان به دنیا آمد"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

generated_ids = model.generate(input_ids,
                                    num_beams=7,
                                    num_return_sequences=7,
#                                     top_k=10,
#                                     top_p=.2,
#                                     temperature=0.95,

                                    max_length=200)
for generated in generated_ids:
    generated_text = tokenizer.decode(generated,skip_special_tokens=True)
print(generated_text)

In [ ]:
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input'], truncation=True, padding='max_length', max_length=564)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target'], truncation=True, padding='max_length', max_length=64)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings


In [ ]:
dataset = dataset.map(convert_to_features, batched=True, remove_columns=dataset['train'].column_names)

columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']

dataset.set_format(type='torch', columns=columns)



Map:   0%|          | 0/47054 [00:00<?, ? examples/s]

Map:   0%|          | 0/11764 [00:00<?, ? examples/s]

In [ ]:
# from transformers import default_data_collator
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,Seq2SeqTrainingArguments, Seq2SeqTrainer
# data_collator = default_data_collator
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
from rouge_score import rouge_scorer, scoring

def add_newline_to_end_of_each_sentence(x):
        return "\n".join(nltk.sent_tokenize(x))

def extract_rouge_mid_statistics(dct):
        new_dict = {}
        for k1, v1 in dct.items():
            mid = v1.mid
            new_dict[k1] = {stat: round(getattr(mid, stat), 4) for stat in ["precision", "recall", "fmeasure"]}
        return new_dict

def calculate_rouge(
        pred_lns,
        tgt_lns,
        use_stemmer=True,
        rouge_keys=["rouge1", "rouge2", "rougeL", "rougeLsum"],
        return_precision_and_recall=False,
        bootstrap_aggregation=True,
        newline_sep=True,
       rouge_lang='bengali'
    ):

        logger.info("Rouge lang: " + str(rouge_lang))
        scorer = rouge_scorer.RougeScorer(
            rouge_keys, lang=rouge_lang,
            use_stemmer=use_stemmer
        )
        aggregator = scoring.BootstrapAggregator()
        for pred, tgt in zip(tgt_lns, pred_lns):
            # rougeLsum expects "\n" separated sentences within a summary
            if newline_sep:
                pred = add_newline_to_end_of_each_sentence(pred)
                tgt = add_newline_to_end_of_each_sentence(tgt)
            scores = scorer.score(pred, tgt)
            aggregator.add_scores(scores)

        if bootstrap_aggregation:
            result = aggregator.aggregate()
            if return_precision_and_recall:
                return extract_rouge_mid_statistics(result)  # here we return dict
            else:
                results_precision = {k: round(v.mid.precision * 100, 4) for k, v in result.items()}
                results_recall = {k: round(v.mid.recall * 100, 4) for k, v in result.items()}
                results_fmeasure = {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}

            return {
                    "rouge1_precision": results_precision.get('rouge1'),
                    "rouge1_recall": results_recall.get('rouge1'),
                    "rouge1_fmeasure": results_fmeasure.get('rouge1'),

                    "rouge2_precision": results_precision.get('rouge2'),
                    "rouge2_recall": results_recall.get('rouge2'),
                    "rouge2_fmeasure": results_fmeasure.get('rouge2'),

                    "rougeL_precision": results_precision.get('rougeL'),
                    "rougeL_recall": results_recall.get('rougeL'),
                    "rougeL_fmeasure": results_fmeasure.get('rougeL'),

                    "rougeLsum_precision": results_precision.get('rougeLsum'),
                    "rougeLsum_recall": results_recall.get('rougeLsum'),
                    "rougeLsum_fmeasure": results_fmeasure.get('rougeLsum'),
                }
#                 return {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}

        else:
            return aggregator._scores  # here we return defaultdict(list)


metric_fn = calculate_rouge

In [ ]:
import numpy as np
from datasets import load_metric
# from sacrebleu.metrics import BLEU
# bleu = BLEU(max_ngram_order=4)
# bleu1 = BLEU(max_ngram_order=1)
# bleu2 = BLEU(max_ngram_order=2)
# bleu3 = BLEU(max_ngram_order=3)

from torchmetrics import SacreBLEUScore

bleu_metric1 = SacreBLEUScore(n_gram=1)
bleu_metric2 = SacreBLEUScore(n_gram=2)
bleu_metric3 = SacreBLEUScore(n_gram=3)
bleu_metric4 = SacreBLEUScore(n_gram=4)

# from nlgeval import NLGEval
# nlgeval = NLGEval()  # loads the models
# metrics_dict = nlgeval.compute_metrics(references, hypothesis)

# metric = load_metric("sacrebleu")
meteor = load_metric('meteor')

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)


    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


    decoded_preds_rouge = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels_rouge = tokenizer.batch_decode(labels, skip_special_tokens=True)
   # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result_rouge = metric_fn(decoded_preds_rouge, decoded_labels_rouge)

#     result_bleu = metric.compute(predictions=decoded_preds, references=decoded_labels)
#     result_bleu4 = round(bleu.corpus_score(decoded_preds, decoded_labels).score, 4)
#     result_bleu1 = round(bleu1.corpus_score(decoded_preds, decoded_labels).score, 4)
#     result_bleu2 = round(bleu2.corpus_score(decoded_preds, decoded_labels).score, 4)
#     result_bleu3 = round(bleu3.corpus_score(decoded_preds, decoded_labels).score, 4)
    result_bleu1 = bleu_metric1(decoded_preds, decoded_labels)
    result_bleu2 = bleu_metric2(decoded_preds, decoded_labels)
    result_bleu3 = bleu_metric3(decoded_preds, decoded_labels)
    result_bleu4 = bleu_metric4(decoded_preds, decoded_labels)

    meteor_result = meteor.compute(predictions=decoded_preds, references=decoded_labels)

#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]

    result = result_rouge
#     result["bleu-1"] = result_bleu1
#     result["bleu-2"] = result_bleu2
#     result["bleu-3"] = result_bleu3
#     result["bleu-4"] = result_bleu4

    result["bleu-1"] = round((torch.FloatTensor(result_bleu1).item() * 100), 4)
    result["bleu-2"] = round((torch.FloatTensor(result_bleu2).item() * 100), 4)
    result["bleu-3"] = round((torch.FloatTensor(result_bleu3).item() * 100), 4)
    result["bleu-4"] = round((torch.FloatTensor(result_bleu4).item() * 100), 4)
#     result["sacrebleu"] =  round(result_bleu['score'], 4)


    result["meteor"] = round(meteor_result["meteor"],4)
#     metrics_dict = nlgeval.compute_metrics(decoded_preds, decoded_labels)
#     result['nlg'] = metrics_dict

#     result["gen_len"] = np.mean(prediction_lens)
#     result = {k: round(v,4) for k, v in result.items()}
    return result

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `SacreBLEUScore` from `torchmetrics` was deprecated and will be removed in 2.0. Import `SacreBLEUScore` from `torchmetrics.text` instead.
  _future_warning(
<ipython-input-15-9a601ee5df84>:21: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  meteor = load_metric('meteor')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments

# set training arguments - Feel free to adapt it
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/mt5-pquad",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    save_strategy="epoch",
    save_steps=4000,
    #eval_steps=1000,
    overwrite_output_dir=True,
    save_total_limit=3,
    load_best_model_at_end=True,
    # push_to_hub=False
    #fp16=True,
)

In [ ]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    # data_collator=T2TDataCollator(),
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics
)

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.418500,0.371084


TrainOutput(global_step=11764, training_loss=0.35347661287838894, metrics={'train_runtime': 4796.0151, 'train_samples_per_second': 9.811, 'train_steps_per_second': 2.453, 'total_flos': 2.740664141881344e+16, 'train_loss': 0.35347661287838894, 'epoch': 1.0})

In [ ]:
# trainer.evaluate(eval_dataset=dataset['test'])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

prompt = "answer: اصفهان context: نگار در اصفهان به دنیا آمد"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids,
                                    num_beams=7,
                                    num_return_sequences=7,
#                                     top_k=10,
#                                     top_p=.2,
#                                     temperature=0.95,

                                    max_length=200)
for generated in generated_ids:
    generated_text = tokenizer.decode(generated,skip_special_tokens=True)
print(generated_text)

نگار در کدام شهر به دنیا آمده است؟


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

prompt = "answer: طور context: اي بني اسرائيل ما شما را از (چنگال) دشمنتان نجات داديم، و در طرف راست كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل كرديم.د"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids,
                                    num_beams=7,
                                    num_return_sequences=7,
#                                     top_k=10,
#                                     top_p=.2,
#                                     temperature=0.95,

                                    max_length=200)
for generated in generated_ids:
    generated_text = tokenizer.decode(generated,skip_special_tokens=True)
print(generated_text)

در طرف راست چه کسی ما شما را از (چنگال) دشمنتان نجات دادند؟
